# Modules

In [87]:
# Modules
import scipy as sp
from scipy import sparse, signal, stats
from astropy import convolution as conv
import numpy as np
from numpy.fft import fft, ifft
import matplotlib.pyplot as plt
from matplotlib import colors as clrs
import pandas as pd 
import time
import datetime
import calendar
import seaborn as sns
from copy import deepcopy

import multiprocessing as mp
from multiprocessing.dummy import Pool as ThreadPool

%matplotlib inline 

import sys
sys.path.insert(1, r'functions')  # add to pythonpath
from detect_peaks import detect_peaks


# Color definitions
ClrS = (0.74, 0.00, 0.00)
ClrN = (0.20, 0.56, 1.00)

Clr = [(0.00, 0.00, 0.00),
      (0.31, 0.24, 0.00),
      (0.43, 0.16, 0.49),
      (0.32, 0.70, 0.30),
      (0.45, 0.70, 0.90),
      (1.00, 0.82, 0.67)]

# Font Size

font = {'family': 'sans-serif',
        'weight': 'normal',
        'size'   : 21}

plt.rc('font', **font)



# Reading data

In [96]:
#REF_Dat = pd.read_csv('input_data/SC_SP_RG_DB_KM_group_areas_by_day.csv', quotechar = '"', encoding = 'ansi',header = 0)

# import dask.dataframe as dd
# df = dd.read_csv('C:/Users/usuario/Documents/Fisica/Investigacion/27-Boulder/ADFSSNGit/scripts/output/Run-2018-12-18-SSN3-2/*.csv')

import glob
import pandas as pd
path ='D:/Mis Documentos/AAResearch/Codes/ADFSSN/runcsvs/' # use your path
# csvFiles = glob.glob(path + "/*.csv")


PL = [100]
PH = [60, 75, 90]
QD = [0]
AD = [0.6, 0.75, 0.9]

# Reading reference
df = pd.read_csv(path + 'A_O_NB1_Observer_ADF.csv', header=0).drop_duplicates(subset='Observer', keep='last')
comObs = df['Observer'].values.copy()                

# First pass to identify common observers
for ipl, pl in enumerate(PL):
    for iph, ph in enumerate(PH):
        for iqd, qd in enumerate(QD):
            for iad, ad in enumerate(AD):
                
                df = pd.read_csv(path + 'A_D_NB1_PL' + str(pl) + '_PH' + str(ph) + '_QD' + str(qd) + '_AD' + str(ad) + '_Observer_ADF.csv', header=0).drop_duplicates(subset='Observer', keep='last')
                comObs = np.intersect1d(comObs, df['Observer'].values, assume_unique=False, return_indices=False)

                
## Creating multi-index dataframe from reference

# Reading reference and keeping only common observers
df = pd.read_csv(path + 'A_O_NB1_Observer_ADF.csv', header=0).drop_duplicates(subset='Observer', keep='last').set_index('Observer')
df = df.loc[comObs,:]

# Create Multi-index
MxInx = pd.MultiIndex.from_product([[0],[0],[0],[0],comObs], names=['PL', 'PH', 'QD', 'AD','Observer'])
MxDf = pd.DataFrame(df.values, index=MxInx, columns = df.columns)

# Append Runs

# First pass to identify common observers
for ipl, pl in enumerate(PL):
    for iph, ph in enumerate(PH):
        for iqd, qd in enumerate(QD):
            for iad, ad in enumerate(AD):
                
                df = pd.read_csv(path + 'A_D_NB1_PL' + str(pl) + '_PH' + str(ph) + '_QD' + str(qd) + '_AD' + str(ad) + '_Observer_ADF.csv', header=0).drop_duplicates(subset='Observer', keep='last').set_index('Observer')
                print(df.shape)
                df = df.loc[comObs,:]
                print(df.shape)
                
                MxInx = pd.MultiIndex.from_product([[pl],[ph],[qd],[ad],comObs], names=['PL', 'PH', 'QD', 'AD','Observer'])
                tmpDf = pd.DataFrame(df.values, index=MxInx, columns = df.columns)
                                
                MxDf = MxDf.append(tmpDf)
                
                

(208, 49)
(208, 49)
(208, 49)
(208, 49)
(208, 49)
(208, 49)
(208, 49)
(208, 49)
(208, 49)
(208, 49)
(208, 49)
(208, 49)
(208, 49)
(208, 49)
(208, 49)
(208, 49)
(208, 49)
(208, 49)


In [118]:
# PL = [100]
# PH = [60, 75, 90]
# QD = [0]
# AD = [0.6, 0.75, 0.9]

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(MxDf.loc[(PL,PH,QD,AD,318),'mreSth'])



[0.19 0.25 0.19 0.26 0.19 0.19 0.23 0.17 0.19]


In [120]:
MxDf['mreSth'].min(level='Observer')

Observer
318    0.170000
321         NaN
322         NaN
323   -0.420000
324   -0.360000
325   -0.250000
327   -0.550000
328   -0.340000
329   -0.440000
330   -0.360000
333   -0.120000
334   -0.380000
335   -0.510000
336   -0.480000
337   -0.360000
339   -0.500000
341   -0.170000
342   -0.330000
343   -0.560000
344    0.020000
345   -0.220000
346   -0.090000
347         inf
348    0.900000
349   -0.380000
350   -0.310000
351   -0.110000
352   -0.140000
353   -0.130000
354   -0.450000
         ...   
590   -0.650000
591   -0.120000
592   -0.230000
595   -0.310000
596   -0.230000
598   -0.370000
599   -0.350000
600   -0.290000
601   -0.300000
602   -0.310000
604   -0.180000
605    0.040000
607   -0.360000
612   -0.350000
613   -0.640000
615   -0.440000
616   -0.330000
617   -0.140000
618   -0.170000
620   -0.110000
621   -0.230000
622   -0.540000
625   -0.390000
626   -0.330000
627   -0.360000
628   -0.440000
629   -0.490000
630   -0.300000
631   -0.370000
632   -0.230000
Name: mreSth, L

In [125]:
import glob
import pandas as pd
path ='C:/Users/usuario/Documents/Fisica/Investigacion/27-Boulder/ADFSSNGit/scripts/output/Run-2018-12-18-SSN3-2' # use your path
allFiles = glob.glob(path + "/*.csv")
# dfold = 0

for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, usecols = [0,1,6,7,8,9,10,11], header=0)
#     print(df)
    df = np.array(df)
    print(df[0,0:5])
    print(np.min(df[0,2:5]))
    print(np.min(df[0,2:5]-df[0,2:5]))
    
#     data = np.append(df)
    
#     df = pd.DataFrame(df)
#     print(df)
#     dfold = df
# index=data[:,0]

# print(df)

[318 'Spoerer' 0.19 0.32 1.19]
0.19
0.0
[318 'Spoerer' 0.25 0.37 1.25]
0.25
0.0
[318 'Spoerer' 0.19 0.32 1.19]
0.19
0.0
[318 'Spoerer' 0.26 0.38 1.26]
0.26
0.0
[318 'Spoerer' 0.19 0.32 1.19]
0.19
0.0
[318 'Spoerer' 0.19 0.32 1.19]
0.19
0.0
[318 'Spoerer' 0.23 0.36 1.23]
0.23
0.0
[318 'Spoerer' 0.17 0.29 1.17]
0.17
0.0
[318 'Spoerer' 0.19 0.32 1.19]
0.19
0.0
